### In this notebook we will make the data preprocessing for the MeteoLibre dataset.

1. First the h5 dataset (not a lot to do here the data is already processed and ready to be used). 

The list of files already exist nothing to be done in reality

2. Second the ground station dataset

3. The two bufr file to be preprocess (and created via the bufr_preprocessing.py script)



In [3]:
import pandas as pd
import os

## 2. Ground station

In [ ]:
# 2. we first take a look at the data.gouv files to get the url of the historical data


file_ressources = "../data/datagouv/2ad89e9d0b014ad0fc3b605dc69b9d41.parquet"

ressources = pd.read_parquet(file_ressources)
ressources.head()

In [2]:
id_datastations = "6569b4473bedf2e7abad3b72"
ressources_meteofrance = ressources[ressources["dataset.id"] == id_datastations]

In [ ]:
ressources_meteofrance["description"].dropna().values

In [4]:
# drop false
values = ressources_meteofrance["description"].dropna().str.contains("2024-2025").values

idx = ressources_meteofrance["description"].dropna()[values].index

In [5]:
data_to_download = ressources_meteofrance.loc[idx]

In [ ]:
data_to_download[["description", "url"]].head().values


In [ ]:
data_to_download[['id', 'url', 'title',
       'description', 'filetype', 'type', 'format', 'mime', 'filesize']]

In [ ]:
# loop over data_to_download and download the data from dataset.url col
for url in data_to_download["url"]:
    !wget {url} -O ../data/groundstations/{url.split("/")[-1]}

In [45]:
## now for each files in data/groundstations we unzip the .gz files read it, create a parquet file
import os

list_files = os.listdir("../data/groundstations")

# only .gz file
#list_files = [f for f in list_files if f.endswith(".gz")]
list_files = [f for f in list_files if f.endswith(".csv")]

# we loop over the file and unzip it read it save it into parquet format and delete the .gz file
for file in list_files:
    try:
        print(file)
        #!gzip -d ../data/groundstations/{file}
        
        # remove the last .gz 
        # file = file[:-3]

        # read the csv file
        df = pd.read_csv(f"../data/groundstations/{file}", sep=";")
        
        # save it into parquet format
        df.to_parquet(f"../data/groundstations_parquet/{file.split('.')[0]}.parquet")
        
        # delete the .gz file
        # os.remove(f"../data/groundstations/{file}")
    except Exception as e:
        
        print(e)

H_82_latest-2024-2025.csv
H_71_latest-2024-2025.csv
H_08_latest-2024-2025.csv
H-COMP_42_latest-2024-2025.csv
H_30_latest-2024-2025.csv
H_49_latest-2024-2025.csv
H-COMP_03_latest-2024-2025.csv
H_87_latest-2024-2025.csv
H_74_latest-2024-2025.csv
H-COMP_06_latest-2024-2025.csv
H_35_latest-2024-2025.csv
H-COMP_09_latest-2024-2025.csv
H-COMP_70_latest-2024-2025.csv
H_43_latest-2024-2025.csv
H-COMP_48_latest-2024-2025.csv
H-COMP_31_latest-2024-2025.csv
H_02_latest-2024-2025.csv
H_88_latest-2024-2025.csv
H_46_latest-2024-2025.csv
H_07_latest-2024-2025.csv
H-COMP_34_latest-2024-2025.csv
H-COMP_94_latest-2024-2025.csv
H-COMP_67_latest-2024-2025.csv
H_54_latest-2024-2025.csv
H-COMP_26_latest-2024-2025.csv
H_15_latest-2024-2025.csv
H_28_latest-2024-2025.csv
H_51_latest-2024-2025.csv
H_975_latest-2024-2025.csv
H_986_latest-2024-2025.csv
H-COMP_91_latest-2024-2025.csv
H-COMP_62_latest-2024-2025.csv
H_69_latest-2024-2025.csv
H_10_latest-2024-2025.csv
H-COMP_23_latest-2024-2025.csv
H_66_latest-2024-2

In [47]:
data_parquet.head()

,NUM_POSTE,NOM_USUEL,LAT,LON,ALTI,AAAAMMJJHH,RR1,QRR1,DRR1,QDRR1,...,INS,QINS,INS2,QINS2,TLAGON,QTLAGON,TVEGETAUX,QTVEGETAUX,ECOULEMENT,QECOULEMENT
0,14047002,BAYEUX,49.274833,-0.712333,60,2024010100,0.4,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14047002,BAYEUX,49.274833,-0.712333,60,2024010101,0.2,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14047002,BAYEUX,49.274833,-0.712333,60,2024010102,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14047002,BAYEUX,49.274833,-0.712333,60,2024010103,0.2,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14047002,BAYEUX,49.274833,-0.712333,60,2024010104,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

dir_ = "../data/groundstations_parquet"

# we list all the parquet file in ../data/groundstations_parquet
list_parquet = os.listdir(dir_)

# we take only the parquet file that start with H
list_parquet = [file for file in list_parquet if ".parquet" in file]


for file in list_parquet:
       
       print(file)
       
       data_parquet = pd.read_parquet(dir_ + "/" + file)

       describe_info = data_parquet.describe()

       # we keep only the columns with more that half non NaN values
       idx_columns = describe_info.loc["count"] > len(data_parquet) / 2

       columns_taken = idx_columns.index[idx_columns.values]

       columns_taken = ['NUM_POSTE', 'LAT', 'LON', 'ALTI', 'AAAAMMJJHH', 'RR1', 'QRR1', 'FF',
              'QFF', 'DD', 'QDD', 'FXY', 'QFXY', 'DXY', 'QDXY', 'HXY', 'QHXY', 'FXI',
              'QFXI', 'DXI', 'QDXI', 'HXI', 'QHXI', 'FXI3S', 'QFXI3S', 'HFXI3S',
              'QHFXI3S', 'T', 'QT', 'TN', 'QTN', 'HTN', 'QHTN', 'TX', 'QTX', 'HTX',
              'QHTX', 'DG', 'QDG']

       custom_df = data_parquet[list(columns_taken) + ["NOM_USUEL"]]
       
       # save the custom_df 
       custom_df.to_parquet("../data/groundstations_filter/{}".format(file))



H_34_latest-2024-2025.parquet
H_03_latest-2024-2025.parquet
H_87_latest-2024-2025.parquet
H-COMP_22_latest-2024-2025.parquet
H-COMP_15_latest-2024-2025.parquet
H-COMP_91_latest-2024-2025.parquet
H_69_latest-2024-2025.parquet
H_07_latest-2024-2025.parquet
H_30_latest-2024-2025.parquet
H_83_latest-2024-2025.parquet
H-COMP_48_latest-2024-2025.parquet
H-COMP_11_latest-2024-2025.parquet
H-COMP_26_latest-2024-2025.parquet
H_65_latest-2024-2025.parquet
H_52_latest-2024-2025.parquet
H-COMP_44_latest-2024-2025.parquet
H_38_latest-2024-2025.parquet
H_56_latest-2024-2025.parquet
H_61_latest-2024-2025.parquet
H-COMP_19_latest-2024-2025.parquet
H-COMP_40_latest-2024-2025.parquet
H-COMP_77_latest-2024-2025.parquet
H_988_latest-2024-2025.parquet
H-COMP_69_latest-2024-2025.parquet
H-COMP_07_latest-2024-2025.parquet
H-COMP_30_latest-2024-2025.parquet
H_48_latest-2024-2025.parquet
H_11_latest-2024-2025.parquet
H_26_latest-2024-2025.parquet
H_95_latest-2024-2025.parquet
H-COMP_34_latest-2024-2025.parquet

KeyError: "None of [Index(['NUM_POSTE', 'LAT', 'LON', 'ALTI', 'AAAAMMJJHH', 'RR1', 'QRR1', 'FF',\n       'QFF', 'DD', 'QDD', 'FXY', 'QFXY', 'DXY', 'QDXY', 'HXY', 'QHXY', 'FXI',\n       'QFXI', 'DXI', 'QDXI', 'HXI', 'QHXI', 'FXI3S', 'QFXI3S', 'HFXI3S',\n       'QHFXI3S', 'T', 'QT', 'TN', 'QTN', 'HTN', 'QHTN', 'TX', 'QTX', 'HTX',\n       'QHTX', 'DG', 'QDG', 'NOM_USUEL'],\n      dtype='object')] are in the [columns]"

In [78]:
# now we can save this dataframe
custom_df.to_parquet("../data/groundstations_parquet/custom_df.parquet")

204

In [76]:
columns_taken

Index(['NUM_POSTE', 'LAT', 'LON', 'ALTI', 'AAAAMMJJHH', 'RR1', 'QRR1', 'FF',
       'QFF', 'DD', 'QDD', 'FXY', 'QFXY', 'DXY', 'QDXY', 'HXY', 'QHXY', 'FXI',
       'QFXI', 'DXI', 'QDXI', 'HXI', 'QHXI', 'FXI3S', 'QFXI3S', 'HFXI3S',
       'QHFXI3S', 'T', 'QT', 'TN', 'QTN', 'HTN', 'QHTN', 'TX', 'QTX', 'HTX',
       'QHTX', 'DG', 'QDG'],
      dtype='object')

In [62]:
data_parquet.columns

Index(['NUM_POSTE', 'NOM_USUEL', 'LAT', 'LON', 'ALTI', 'AAAAMMJJHH', 'RR1',
       'QRR1', 'DRR1', 'QDRR1',
       ...
       'INS', 'QINS', 'INS2', 'QINS2', 'TLAGON', 'QTLAGON', 'TVEGETAUX',
       'QTVEGETAUX', 'ECOULEMENT', 'QECOULEMENT'],
      dtype='object', length=204)